### 키워드 추출 최종 아웃풋

[빈도기반]

keyword_list_after_cleaning.csv -> long type

keyword_list_wide_format.csv -> wide type

[TF-IDF]

keyword_list_after_cleaning_tfidf.csv -> long type

keyword_list_wide_format_tfidf.csv -> wide type

In [23]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [24]:
# 전처리, 명사추출, 키워드추출, 불용어후처리까지 마친 데이터
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_after_kw_cleaning.csv')
comments_df = pd.read_csv(file_path)

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df = comments_df.copy()
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [25]:
sample_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,"[중독, 진심, 노래, 의문, 엔믹스]"
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,"[기도, 아쉬움, 대쉬, 금도, 대중성, 노래]"
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,"[본부장, 엘범, 천만, 클릭, 연락, 연결, 취해, 피씨, 뮤비, 노래, 뷰수, ..."
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,[노래]
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,"[걸그룹, 멤버, 빵빵, 노력, 묵자, 얼마나, 세대, 외모췍, 더더, 리더, 해원]"


# 키워드 추출

In [26]:
import time
from collections import defaultdict

def extract_keywords_frequency(series_word_list, top_n=30, min_df=1, existing_keywords=None):
    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)

    # 기존 키워드 제외하고 새로운 키워드만 추출
    new_keywords_with_weights = [(word, count) for word, count in sorted_words if word not in existing_keywords]
    
    # 상위 n개 키워드만 추출 (중복 제외 후)
    top_keywords_with_weights = new_keywords_with_weights[:top_n]

    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    if total_count > 0:
        keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]
    else:
        keywords_with_weights = [(word, 0) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights


In [27]:
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1, existing_keywords=None):

    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # TF-IDF 벡터화 (min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[::-1]  # 높은 TF-IDF 순으로 정렬
    top_keywords = [feature_names[idx] for idx in top_n_indices if feature_names[idx] not in existing_keywords]

    # 상위 n개 키워드만 추출
    top_keywords = top_keywords[:top_n]

    # 각 단어의 TF-IDF 점수 추출
    top_n_indices = [i for i, word in enumerate(feature_names) if word in top_keywords]
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = np.array(weights) / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights


# 각 레벨 별로 키워드 추출

In [28]:
## keyword_list 초기화 및 설정
keyword_list = pd.DataFrame(columns=['Group', 'Title', 'keyword'])

# 전체 리스트에서 배제할 단어
group_keywords = [
    '마그네틱', '안유진', '이서', '단군신화', '혜진', '민지', '카리나', '레이', '류진', 
    '마고', '하니', '단월드', '피프티', '윈터', '정신병원', '스테이씨', '어도어', 
    '원영', '디토', '뉴진스', '사이비', '아일릿', '장원영', '다이브', '버니즈', 
    '에스파', '해린', '가을', '유진', '닝닝', '방탄소년단', '이브', '수가', '리아', 
    '여자친구', '다니엘', '아이브', '오엠지', '르세라핌', '예지', '하이브', '침착맨', 
    '방시혁', '혜인', '민희진', '지젤', '엔믹스', '박지원', '리즈'
]

# 단순 빈도 기반 키워드 추출
keywords, weights = extract_keywords_frequency(sample_df['word_list'], top_n=30, min_df=5, existing_keywords=group_keywords)
keyword_list['keyword'] = keywords
keyword_list['weights'] = weights
keyword_list['Group'] = 'All'
keyword_list['Title'] = 'All'

## 각 그룹에 대한 키워드 추출
# Group 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()

# 기존 keyword_list에서 추출된 키워드와 중복되지 않도록 새로운 키워드 추출
# existing_keywords = keyword_list['keyword'].tolist()

# 그룹별로 새로운 키워드 추출
for group in groups:
    word_list = sample_df[sample_df['Group'] == group]['word_list']
    keywords, weights = extract_keywords_frequency(word_list, top_n=30, min_df=5)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = 'All'
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')

## 각 뮤비에 대한 키워드 추출
# Title 열에 속한 그룹들의 리스트
titles = sample_df['Title'].unique()

# MV별로 새로운 키워드 추출
for title in titles:
    group = sample_df[sample_df['Title'] == title]['Group'].iloc[0] # 해당 타이틀에 속한 그룹 조회
    word_list = sample_df[sample_df['Title'] == title]['word_list'] # 댓글 시리즈 가져오기
    keywords, weights = extract_keywords_frequency(word_list, top_n=30, min_df=3)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = title
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')

# 인덱스 초기화 및 csv 익스포트
keyword_list = keyword_list.reset_index(drop=True)
keyword_list.to_csv('keyword_list_after_cleaning.csv', encoding='utf-8-sig', index=False)

648060개 댓글 키워드 추출에 걸린 시간: 0.7090978622436523 초

23511개 댓글 키워드 추출에 걸린 시간: 0.018292903900146484 초

50699개 댓글 키워드 추출에 걸린 시간: 0.03465986251831055 초

249967개 댓글 키워드 추출에 걸린 시간: 0.33467888832092285 초

45376개 댓글 키워드 추출에 걸린 시간: 0.04183793067932129 초

7425개 댓글 키워드 추출에 걸린 시간: 0.004938364028930664 초

136993개 댓글 키워드 추출에 걸린 시간: 0.10950493812561035 초

33181개 댓글 키워드 추출에 걸린 시간: 0.024754047393798828 초

7846개 댓글 키워드 추출에 걸린 시간: 0.005994081497192383 초

69125개 댓글 키워드 추출에 걸린 시간: 0.0479581356048584 초

23937개 댓글 키워드 추출에 걸린 시간: 0.016405820846557617 초

4339개 댓글 키워드 추출에 걸린 시간: 0.003824949264526367 초

2622개 댓글 키워드 추출에 걸린 시간: 0.002006053924560547 초

551개 댓글 키워드 추출에 걸린 시간: 0.00039196014404296875 초

1442개 댓글 키워드 추출에 걸린 시간: 0.001104116439819336 초

1406개 댓글 키워드 추출에 걸린 시간: 0.0008418560028076172 초

1416개 댓글 키워드 추출에 걸린 시간: 0.0009510517120361328 초

1916개 댓글 키워드 추출에 걸린 시간: 0.00131988525390625 초

1608개 댓글 키워드 추출에 걸린 시간: 0.0010859966278076172 초

570개 댓글 키워드 추출에 걸린 시간: 0.00041103363037109375 초

3550개 댓글 키워드 추출에 걸린 시간: 0.002923

In [29]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

#keyword_list
keyword_list[:100]

,Group,Title,keyword,weights
0,All,All,노래,"(노래, 0.23492009788337412)"
1,All,All,뮤비,"(뮤비, 0.10313337630557345)"
2,All,All,사랑해,"(사랑해, 0.05322627217761341)"
3,All,All,컨셉,"(컨셉, 0.038112099213374676)"
4,All,All,멤버,"(멤버, 0.03763959901649959)"
5,All,All,응원,"(응원, 0.034475014364589315)"
6,All,All,화이팅,"(화이팅, 0.03235459681441534)"
7,All,All,한국,"(한국, 0.03219709674879031)"
8,All,All,가사,"(가사, 0.029122928801220332)"
9,All,All,데뷔,"(데뷔, 0.028242095100872958)"


In [30]:
# 'Group'과 'Title'로 그룹화한 후, Keyword를 와이드 포맷으로 변환
keyword_list['Keyword_Index'] = keyword_list.groupby(['Group', 'Title']).cumcount() + 1

# 'Keyword_Index'를 열로 피봇하여 와이드 포맷으로 변환
df_wide_format = keyword_list.pivot(index=['Group', 'Title'], columns='Keyword_Index', values='keyword')

# 결과 출력
df_wide_format.to_csv('keyword_list_wide_format.csv', encoding='utf-8-sig', index=False)
df_wide_format

Keyword_Index                                                1      2      3   \
Group       Title                                                               
All         All                                              노래     뮤비    사랑해   
FIFTY FIFTY All                                              노래    피프티     대표   
            Cupid                                            노래     대표    피프티   
            Higher                                           노래     음색    피프티   
            Lovin' Me                                        노래    피프티     뮤비   
            SOS                                              노래    피프티     응원   
            Starry Night                                     노래    피프티     응원   
ITZY        All                                              노래     류진     채령   
            BET ON ME                                        노래     류진    사랑해   
            BORN TO BE                                       노래     리아    사랑해   
            Boys Like You                                    노래     리아    테일러   
            CAKE                                             노래    케이크    사랑해   
            Cheshire                                         노래    사랑해     체셔   
            ICY                                              노래    한국인     뮤비   
            LOCO                                             노래    사랑해     예지   
            Mr. Vampire                                      노래     리아     류진   
            None of My Business                              노래     류진     여름   
            Not Shy                                          노래     류진    한국인   
            SNEAKERS                                         노래    사랑해   스니커즈   
            SWIPE                                            노래    타이틀     뮤비   
            UNTOUCHABLE                                      노래     컴백     리아   
            Voltage                                          노래     일본     한국   
            WANNABE                                          노래     류진    한국인   
            달라달라(DALLA DALLA)                          노래     채령    한국인   
            마.피.아. In the morning                         노래    마피아     가사   
IVE         Accendio                                       아센디오    아이브     뮤비   
            After LIKE                                      아이브     노래     가을   
            All                                             아이브     노래     리즈   
            All Night (Feat. Saweetie)                      아이브     노래    사랑해   
            Baddie                                          아이브     노래    사랑해   
            ELEVEN                                          아이브     노래    사랑해   
            Either Way                                      아이브     노래    사랑해   
            I AM                                            아이브     노래     리즈   
            Kitsch                                           리즈     노래    아이브   
            LOVE DIVE                                       아이브     노래    다이브   
            Off The Record                                  아이브     노래    사랑해   
            해야 (HEYA)                                      중국     한국    아이브   
Kep1er      All                                              노래    케플러    사랑해   
            Giddy                                            노래    케플러    사랑해   
            Grand Prix                                      케플러    사랑해     노래   
            I do! Do you                                     노래    케플러     한국   
            Straight Line                                   케플러     노래    사랑해   
            Sugar Rush                                      케플러     노래    NaN   
            Up!                                              노래    케플러    사랑해   
            WA DA DA                                         노래    케플러     데뷔   
            We Fresh                                        케플러     노래    사랑해   
            Wing Wing                                        노래    케플러     일본   
LE SSERAFIM ANTIFRAGILE        

# Another Ways...

In [32]:
## keyword_list 초기화 및 설정
keyword_list_tf = pd.DataFrame(columns=['Group', 'Title', 'keyword'])

# 전체 리스트에서 배제할 단어
group_keywords = [
    '마그네틱', '안유진', '이서', '단군신화', '혜진', '민지', '카리나', '레이', '류진', 
    '마고', '하니', '단월드', '피프티', '윈터', '정신병원', '스테이씨', '어도어', 
    '원영', '디토', '뉴진스', '사이비', '아일릿', '장원영', '다이브', '버니즈', 
    '에스파', '해린', '가을', '유진', '닝닝', '방탄소년단', '이브', '수가', '리아', 
    '여자친구', '다니엘', '아이브', '오엠지', '르세라핌', '예지', '하이브', '침착맨', 
    '방시혁', '혜인', '민희진', '지젤', '엔믹스', '박지원', '리즈'
]

# 단순 빈도 기반 키워드 추출
keywords, weights = extract_keywords_tfidf(sample_df['word_list'], top_n=30, min_df=5, existing_keywords=group_keywords)
keyword_list_tf['keyword'] = keywords
keyword_list_tf['weights'] = weights
keyword_list_tf['Group'] = 'All'
keyword_list_tf['Title'] = 'All'

## 각 그룹에 대한 키워드 추출
# Group 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()

# 기존 keyword_list에서 추출된 키워드와 중복되지 않도록 새로운 키워드 추출
# existing_keywords = keyword_list['keyword'].tolist()

# 그룹별로 새로운 키워드 추출
for group in groups:
    word_list = sample_df[sample_df['Group'] == group]['word_list']
    keywords, weights = extract_keywords_tfidf(word_list, top_n=30, min_df=5)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = 'All'
    
    keyword_list_tf = pd.concat([keyword_list_tf, frac])
    keyword_list_tf.drop_duplicates(subset='keyword', keep='first')

## 각 뮤비에 대한 키워드 추출
# Title 열에 속한 그룹들의 리스트
titles = sample_df['Title'].unique()

# MV별로 새로운 키워드 추출
for title in titles:
    group = sample_df[sample_df['Title'] == title]['Group'].iloc[0] # 해당 타이틀에 속한 그룹 조회
    word_list = sample_df[sample_df['Title'] == title]['word_list'] # 댓글 시리즈 가져오기
    keywords, weights = extract_keywords_tfidf(word_list, top_n=30, min_df=3)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = title
    
    keyword_list_tf = pd.concat([keyword_list_tf, frac])
    keyword_list_tf.drop_duplicates(subset='keyword', keep='first')

# 인덱스 초기화 후 csv 익스포트
keyword_list_tf = keyword_list_tf.reset_index(drop=True)
keyword_list_tf.to_csv('keyword_list_after_cleaning_tfidf.csv', encoding='utf-8-sig', index=False)

# 'Group'과 'Title'로 그룹화한 후, Keyword를 와이드 포맷으로 변환
keyword_list_tf['Keyword_Index'] = keyword_list_tf.groupby(['Group', 'Title']).cumcount() + 1

# 'Keyword_Index'를 열로 피봇하여 와이드 포맷으로 변환
df_wide_format_tf = keyword_list_tf.pivot(index=['Group', 'Title'], columns='Keyword_Index', values='keyword')

# 결과 출력 및 익스포트
df_wide_format_tf.to_csv('keyword_list_wide_format_tfidf.csv', encoding='utf-8-sig', index=False)
df_wide_format_tf

648060개 댓글 키워드 추출에 걸린 시간: 2.365571975708008 초

23511개 댓글 키워드 추출에 걸린 시간: 0.058706045150756836 초

50699개 댓글 키워드 추출에 걸린 시간: 0.11121201515197754 초

249967개 댓글 키워드 추출에 걸린 시간: 0.8691620826721191 초

45376개 댓글 키워드 추출에 걸린 시간: 0.11765193939208984 초

7425개 댓글 키워드 추출에 걸린 시간: 0.015688180923461914 초

136993개 댓글 키워드 추출에 걸린 시간: 0.3562471866607666 초

33181개 댓글 키워드 추출에 걸린 시간: 0.08423829078674316 초

7846개 댓글 키워드 추출에 걸린 시간: 0.02511119842529297 초

69125개 댓글 키워드 추출에 걸린 시간: 0.17873311042785645 초

23937개 댓글 키워드 추출에 걸린 시간: 0.05745196342468262 초

4339개 댓글 키워드 추출에 걸린 시간: 0.015281915664672852 초

2622개 댓글 키워드 추출에 걸린 시간: 0.007776975631713867 초

551개 댓글 키워드 추출에 걸린 시간: 0.0019102096557617188 초

1442개 댓글 키워드 추출에 걸린 시간: 0.004142045974731445 초

1406개 댓글 키워드 추출에 걸린 시간: 0.0040781497955322266 초

1416개 댓글 키워드 추출에 걸린 시간: 0.0039000511169433594 초

1916개 댓글 키워드 추출에 걸린 시간: 0.005121946334838867 초

1608개 댓글 키워드 추출에 걸린 시간: 0.004246950149536133 초

570개 댓글 키워드 추출에 걸린 시간: 0.002207040786743164 초

3550개 댓글 키워드 추출에 걸린 시간: 0.01115703582763

Keyword_Index                                                1      2      3   \
Group       Title                                                               
All         All                                              노래     뮤비    사랑해   
FIFTY FIFTY All                                              노래    피프티     응원   
            Cupid                                            노래     대표    피프티   
            Higher                                           노래     음색     아란   
            Lovin' Me                                        노래    피프티    큐피드   
            SOS                                              노래    피프티     응원   
            Starry Night                                     노래    피프티     응원   
ITZY        All                                              노래     류진    한국인   
            BET ON ME                                        노래    사랑해     가사   
            BORN TO BE                                       노래     리아    사랑해   
            Boys Like You                                    노래     채령     리아   
            CAKE                                             노래    케이크    사랑해   
            Cheshire                                         노래    사랑해     예지   
            ICY                                              노래    한국인     예지   
            LOCO                                             노래    사랑해     예지   
            Mr. Vampire                                      노래     리아     류진   
            None of My Business                              노래     류진    사랑해   
            Not Shy                                          노래     류진    한국인   
            SNEAKERS                                         노래    사랑해   스니커즈   
            SWIPE                                            노래    타이틀     뮤비   
            UNTOUCHABLE                                      노래     리아    사랑해   
            Voltage                                          노래     한국     일본   
            WANNABE                                          노래     류진    한국인   
            달라달라(DALLA DALLA)                          노래    한국인     달라   
            마.피.아. In the morning                         노래    마피아     가사   
IVE         Accendio                                        아이브   아센디오     뮤비   
            After LIKE                                      아이브     노래    사랑해   
            All                                             아이브     노래    사랑해   
            All Night (Feat. Saweetie)                      아이브     노래    사랑해   
            Baddie                                          아이브     노래    사랑해   
            ELEVEN                                          아이브     노래    사랑해   
            Either Way                                      아이브     노래    사랑해   
            I AM                                            아이브     노래    사랑해   
            Kitsch                                           리즈     노래    아이브   
            LOVE DIVE                                       아이브     노래    다이브   
            Off The Record                                  아이브     노래    사랑해   
            해야 (HEYA)                                     아이브     한국     중국   
Kep1er      All                                              노래    케플러    사랑해   
            Giddy                                            노래    케플러    사랑해   
            Grand Prix                                      케플러     노래    사랑해   
            I do! Do you                                     노래    케플러     일본   
            Straight Line                                   케플러     노래    사랑해   
            Sugar Rush                                      케플러     노래    NaN   
            Up!                                              노래    케플러    사랑해   
            WA DA DA                                         노래    케플러     데뷔   
            We Fresh                                        케플러     노래    사랑해   
            Wing Wing                                        노래    케플러     일본   
LE SSERAFIM ANTIFRAGILE        